In [ ]:
!pip install libigl matplotlib numpy polyscope gpytoolbox

## Introduction to Mesh Parameterization: Exercises
In this notebook you will perform the same analysis you did in 101 on more complex meshes, and try your hand at more complicated parameterization techniques. 

### Part 1: Fixed Boundary Parameterization and Distortion Analysis ###
In this folder you are given 2 meshes -- halfbunny.obj and ogre.obj. Load each of these meshes using gpytoolbox and use the code from notebook 101 to perform the analyses in the next few code blocks. 

In [2]:
# Loading required packages
import numpy as np
import polyscope as ps

#### 1.1 halfbunny.obj

In [67]:
# TODO: use gpytoolbox to load in halfbunny.obj
from meshing.io import PolygonSoup
from meshing.mesh import Mesh

soup = PolygonSoup.from_obj("ogre.obj")
mesh = Mesh(soup.vertices, soup.indices)

# TODO: Use the below code copied from notebook 101 to get the boundary and non-boundary edges of the imported mesh
from igl import boundary_loop

bnd = boundary_loop(mesh.faces)
boundary_idxs = list(sorted(bnd))

from igl import map_vertices_to_circle

boundary_positions = map_vertices_to_circle(mesh.vertices, bnd).astype(mesh.vertices.dtype)

# Resort the positions to match the order of boundary_idxs
boundary_positions = boundary_positions[np.argsort(bnd)]

pred_idxs = np.array([i for i in range(mesh.vertices.shape[0]) if i not in boundary_idxs])

# Get edge array
from collections import defaultdict
edges = defaultdict(int)
for f in mesh.faces:
    for i in range(3):
        if f[i] > f[(i+1)%3]:
            edges[(f[(i+1)%3], f[i])] += 1
        else:
            edges[(f[i], f[(i+1)%3])] += 1

# Valid edges are the ones that are shared by two faces
tot_edges = np.array(list(edges.keys()))
valid_edges = np.array([k for k, v in edges.items() if v == 2])
boundary_edges = np.array([k for k, v in edges.items() if v == 1])

In [68]:
# TODO: Copy over the setup_parameterization_matrices() function from notebook 101 and use it to compute
# 1) The Tutte parameterization (all weights of 1)
# 2) The Mean Value weights parameterization (see formula and code for computing the mean value weights from notebook 101)
def setup_parameterization_matrices(vertices, boundary_idxs, edges, weights=1):
    """ Set up the Tutte linear system (laplacian weights of 1)

    vertices (np.ndarray): V x 3 array of vertex positions
    boundary_idxs (np.ndarray): B array of boundary vertex indices
    edges (np.ndarray): E x 2 array of edge indices for weight assignment
    weights (float or np.ndarray): E array of edge weights or scalar value (default 1)

    Returns:
        L (np.ndarray): V x V Laplacian matrix with boundary values set to 0
        Lb (np.ndarray): V x B boundary Laplacian
    """

    # Initialize the laplacian matrix
    L = np.zeros((vertices.shape[0], vertices.shape[0]))

    L[edges[:, 0], edges[:, 1]] = weights
    L[edges[:, 1], edges[:, 0]] = weights

    # Off-diagonal elements are negative and diagonal is the negative sum of the row
    L = np.diag(np.sum(L, axis=1)) - L

    # Boundary weights are positive
    Lb = -L[:, boundary_idxs]

    # Boundary diagonal should be set to 0
    Lb[boundary_idxs, range(len(boundary_idxs))] = 0

    # Set the off-diagonal boundary columns to 0
    Ldiag = np.diag(L).copy()

    L[:, boundary_idxs] = 0
    np.fill_diagonal(L, Ldiag)

    return L, Lb

L, Lb = setup_parameterization_matrices(mesh.vertices, boundary_idxs, valid_edges)

### Solve the Tutte linear system
tutte_solution = np.linalg.solve(L, Lb @ boundary_positions)
tutte_uv = np.zeros((mesh.vertices.shape[0], 2))
tutte_uv[pred_idxs] = tutte_solution[pred_idxs]
tutte_uv[boundary_idxs] = boundary_positions

# ps.init()
# ps.remove_all_structures()
# ps_uv = ps.register_surface_mesh("tutte solution", tutte_solution, mesh.faces, edge_width=1)
# ps_uv = ps.register_surface_mesh("tutte uv", tutte_uv, mesh.faces, edge_width=1)
# ps.show()

In [69]:
# TODO: Copy over the setup_parameterization_matrices() function from notebook 101 and use it to compute
# 1) The Tutte parameterization (all weights of 1)
# 2) The Mean Value weights parameterization (see formula and code for computing the mean value weights from notebook 101)
meanvalues = {}
for fi in range(len(mesh.faces)):
    f = mesh.faces[fi]
    for i in range(3):
        v1 = f[i]
        v2 = f[(i+1)%3]
        v3 = f[(i+2)%3]
        e1 = mesh.vertices[v2] - mesh.vertices[v1]
        e2 = mesh.vertices[v3] - mesh.vertices[v1]

        b1 = mesh.vertices[v1] - mesh.vertices[v2]
        b2 = mesh.vertices[v3] - mesh.vertices[v2]

        assert (v1, v2) not in meanvalues

        alpha = np.arccos(np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2)))
        beta = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))

        meanvalues[(v1, v2)] = (np.tan(alpha/2),
                            np.tan(beta/2),
                            np.linalg.norm(mesh.vertices[v2] - mesh.vertices[v1]))

# For each edge pair, compute the mean value weights
mean_value_weights = np.zeros((len(valid_edges),))
for i, edge in enumerate(valid_edges):
    v1, v2 = edge

    assert (v1, v2) in meanvalues and (v2, v1) in meanvalues

    a1, b2, r1 = meanvalues[(v1, v2)]
    a2, b1, r2 = meanvalues[(v2, v1)]

    assert r1 == r2

    mean_value_weights[i] = (a1 + b1)/r1

L, Lb = setup_parameterization_matrices(mesh.vertices, boundary_idxs, valid_edges, weights=mean_value_weights)

### Solve the Tutte linear system
mean_value_solution = np.linalg.solve(L, Lb @ boundary_positions)

# Bring solution back to final UVs
mean_value_uv = np.zeros_like(mesh.vertices[:, :2])
mean_value_uv[boundary_idxs] = boundary_positions
mean_value_uv[pred_idxs] = mean_value_solution[pred_idxs]

# ps.init()
# ps.remove_all_structures()
# ps_uv = ps.register_surface_mesh("meanvalue", mean_value_uv, mesh.faces, edge_width=1)
# ps.show()

In [70]:
np.save("ogre_tutte.npy", tutte_uv)
np.save("ogre_meanvalue.npy", mean_value_uv)

In [55]:
J = get_jacobian(mesh.vertices, mesh.faces, tutte_uv)

_, S, _ = np.linalg.svd(J)

E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

J = get_jacobian(mesh.vertices, mesh.faces, mean_value_uv)

_, S, _ = np.linalg.svd(J)

E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

Mean conformal energy:  4.729862835966947
Mean isometric energy:  42862123.760993205
Mean conformal energy:  0.016295271
Mean isometric energy:  335560500.0


### Part 2: LSCM and ARAP ###
In this part you will use two more advanced parameterization techniques to flatten the same meshes. 

These two methods are Least Squares Conformal Maps [(LSCM)](https://www.cs.jhu.edu/~misha/Fall09/Levy02.pdf) and As-Rigid-As-Possible Mesh Parameterization [(ARAP)](https://cs.harvard.edu/~sjg/papers/arap.pdf). 

As you will see, these are two examples of **free boundary** methods (though LSCM technically requires two vertices to be pinned), which allows the boundary to move independently to perform the desired distortion minimization. 

#### 2.1 Least Squares Conformal Maps [(LSCM)](https://www.cs.jhu.edu/~misha/Fall09/Levy02.pdf)
As the name implies, LSCM is a **conformal** method, meaning it aims to minimize the conformal (angular) distortion of the parameterization, using a least squares solve. Deriving and computing this method by hand is beyond the scope of this exercise, so we will be using libigl's `lscm()` function to do the computation for us. 

One important note is that LSCM requires two vertices to be pinned in the plane to make the least-squared system well-determined (so there is a unique solution). Technically any two vertices can be chosen, but in practice vertices at the opposite end of a boundary loop are usually the best choice for method performance. The code commented below gives an example of computing the LSCM parameterization using libigl. 

In [65]:
# TODO: use gpytoolbox to load in halfbunny.obj
from meshing.io import PolygonSoup
from meshing.mesh import Mesh

soup = PolygonSoup.from_obj("ogre.obj")
mesh = Mesh(soup.vertices, soup.indices)

# TODO: Use the below code copied from notebook 101 to get the boundary and non-boundary edges of the imported mesh
from igl import boundary_loop

bnd = boundary_loop(mesh.faces)
boundary_idxs = list(sorted(bnd))

from igl import map_vertices_to_circle

boundary_positions = map_vertices_to_circle(mesh.vertices, bnd).astype(mesh.vertices.dtype)

# Resort the positions to match the order of boundary_idxs
boundary_positions = boundary_positions[np.argsort(bnd)]

pred_idxs = np.array([i for i in range(mesh.vertices.shape[0]) if i not in boundary_idxs])

# Get edge array
from collections import defaultdict
edges = defaultdict(int)
for f in mesh.faces:
    for i in range(3):
        if f[i] > f[(i+1)%3]:
            edges[(f[(i+1)%3], f[i])] += 1
        else:
            edges[(f[i], f[(i+1)%3])] += 1

# Valid edges are the ones that are shared by two faces
tot_edges = np.array(list(edges.keys()))
valid_edges = np.array([k for k, v in edges.items() if v == 2])
boundary_edges = np.array([k for k, v in edges.items() if v == 1])

from igl import boundary_loop, lscm

bdry = boundary_loop(mesh.faces)

b = np.array([bdry[0], bdry[int(len(bdry)/2)]], dtype="int")
bc = np.array([[0, 0], [1, 1]], dtype=np.float32)
succ, lscm_uv = lscm(mesh.vertices, mesh.faces, b, bc)

In [47]:
ps.init()
ps.remove_all_structures()
ps_uv = ps.register_surface_mesh("lscm", lscm_uv, mesh.faces, edge_width=1)
ps.show()

In [66]:
def get_jacobian(vs, fs, uvmap):
    """ Get jacobian of mesh given an input UV map

    Args:
        vs (np.ndarray): V x 3 array of vertex positions
        fs (np.ndarray): F x 3 integer array of face indices
        uvmap (np.ndarray): V x 2 array of UV coordinates

    Returns:
        _type_: _description_
    """
    # Visualize distortion
    from igl import grad
    G = np.array(grad(vs, fs).todense())

    # NOTE: currently gradient is organized as X1, X2, X3, ... Y1, Y2, Y3, ... Z1, Z2, Z3 ... reshape to X1, Y1, Z1, ...
    splitind = G.shape[0]//3
    newG = np.zeros_like(G) # F*3 x V
    newG[::3] = G[:splitind]
    newG[1::3] = G[splitind:2*splitind]
    newG[2::3] = G[2*splitind:]

    J = (newG @ uvmap).reshape(-1, 3, 2) # F x 3 x 2
    return J

J = get_jacobian(mesh.vertices, mesh.faces, lscm_uv)

_, S, _ = np.linalg.svd(J)

E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

Mean conformal energy:  0.0008202818835717025
Mean isometric energy:  1155.7294863619238


#### 2.2 As-Rigid-As-Possible Mesh Parameterization [(ARAP)](https://cs.harvard.edu/~sjg/papers/arap.pdf)
The ARAP method aims to minimize isometric distortion (both area and angles), using a non-linear algorithm which alternates between local and global optimization steps. The method requires an initial UV map as input, so we use a harmonic parameterization as an initial guess. 

In [60]:
# TODO: Use the below example code to compute the ARAP parameterization of halfbunny.obj and ogre.obj
from igl import ARAP, boundary_loop, harmonic, map_vertices_to_circle
bnd = boundary_loop(mesh.faces)
bnd_uv = map_vertices_to_circle(mesh.vertices, bnd).astype(mesh.vertices.dtype)
initial_uv = harmonic(mesh.vertices, mesh.faces, bnd, bnd_uv, 1)
arap = ARAP(mesh.vertices, mesh.faces, 2, np.zeros(0), with_dynamics=True)
arap_uv = arap.solve(np.zeros((0,0)), initial_uv)

In [42]:
ps.init()
ps.remove_all_structures()
ps_uv = ps.register_surface_mesh("arap", arap_uv, mesh.faces, edge_width=1)
ps.show()

In [63]:
J = get_jacobian(mesh.vertices, mesh.faces, arap_uv)

_, S, _ = np.linalg.svd(J)

E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

Mean conformal energy:  0.24614305519835683
Mean isometric energy:  4296.2617408718015


In [62]:
np.save("halfbunny_lscm.npy", lscm_uv)
np.save("halfbunny_arap.npy", arap_uv)